In [ ]:
import pandas as pd
import numpy as np
import os
import synapseclient as syn
import synapseutils
import sys
import random
import seaborn as sns
from scipy.stats import ttest_ind as ttest
from scipy.stats import ks_2samp
from sklearn.metrics.pairwise import euclidean_distances
from statsmodels.stats.multitest import fdrcorrection as fdr
from matplotlib import pyplot as plt
from Cycif_data_analysis import per_well_analysis
random.seed(50)

In [ ]:
# Load data from 0907 data
os.chdir('D:/data/MCF10A 090718 data/')
if not os.path.exists('Results'):
    os.mkdir('Results')
datatype = 'Mean'
control_name = 'PBS'
output_prefix = ' '.join([control_name,'as control'])

pooled_expr_data = pd.read_csv(' '.join(['MCF10A 090718 pooled', datatype, 'normalized data.csv']),index_col=0)
pooled_metadata = pd.read_csv('MCF10A 090718 pooled metadata.csv',index_col=0)
pooled_metadata = pooled_metadata[pooled_metadata.condition!='ctrl_0']
pooled_expr_data = pooled_expr_data.loc[pooled_metadata.index]
pooled_expr_data.fillna(0,inplace=True)
pooled_expr_data.columns = sorted(pooled_expr_data.columns)

# Per well analysis
pwa = per_well_analysis(pooled_expr_data,pooled_metadata,batch_col='Plate',drug_col='ligand',dose_col='dose',time_col='time',control_name = control_name, output_prefix=output_prefix)
pwa.make_contrast(batch_wise=True)
pwa.descriptive_analysis()
pwa.contrast_based_distance()
pwa.contrast_based_differential_analysis()
pwa.plot_dist_violinplot('ligand','time')
pwa.make_fc_heamtap_matrix(['ligand','time','Batch'],output_fname=output_prefix)
pwa.make_heatmaps(sorting='simple')
pwa.make_swarm_plot()
pwa.channel_dist_plot()